Just another version

In [1]:
import pandas as pd
import numpy as np
import pydicom
import pickle
import glob
import os

from sklearn.preprocessing import LabelEncoder, PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

from skimage.transform import resize
from awesome_progress_bar import ProgressBar

In [2]:
df = pd.read_csv('data/train.csv', index_col='Patient')
df = df.drop('Percent', axis=1)
df.Sex = LabelEncoder().fit_transform(df.Sex)
df.SmokingStatus = LabelEncoder().fit(['Never smoked', 'Ex-smoker', 'Currently smokes']).transform(df.SmokingStatus)
df.head(3)

,Weeks,FVC,Age,Sex,SmokingStatus
Patient,,,,,
ID00007637202177411956430,-4,2315,79,1,1
ID00007637202177411956430,5,2214,79,1,1
ID00007637202177411956430,7,2061,79,1,1


In [6]:
def create_row(i: str):
    tmp = df.loc[i]
    
    X = tmp.Weeks.values.reshape(-1, 1)
    y = tmp.FVC
    pipe = Pipeline([
        ('prep', PolynomialFeatures(3)),
        ('reg', LinearRegression())
    ]).fit(X, y)
    
    def get(n):
        return tmp[tmp.Weeks == n].FVC[0] if 0 in tmp.Weeks else pipe.predict([[n]])[0]
    
    weeks = df.loc[pat].Weeks
    w = weeks.abs().min()
    w = w if w in weeks.values else -w
    
    return w, [get(w), get(12), get(50)]

In [7]:
data = []
for pat in df.index.unique():
    tmp = df.loc[pat]
    row = [pat]
    w, fvcs = create_row(pat)
    row.extend(fvcs)
    row.extend([
        tmp.SmokingStatus[0],
        tmp.Sex[0],
        tmp.Age[0],
        w,
    ])
    data.append(row)

In [8]:
df = pd.DataFrame.from_dict(data)
df.columns = ['Patient', 'FVC_0', 'FVC_12', 'FVC_50', 'SmokingStatus', 'Sex', 'Age', 'Week']

In [9]:
columns_0 = ['FVC_0', 'SmokingStatus', 'Sex', 'Age']
columns_1 = ['FVC_12', 'FVC_50']
scaler = StandardScaler().fit(df[columns_0])
df[columns_1] = (df[columns_1] - df.FVC_0.mean()) / df.FVC_0.std()
df[columns_0] = scaler.transform(df[columns_0])
df['Week'] /= 133
df.head(3)

,Patient,FVC_0,FVC_12,FVC_50,SmokingStatus,Sex,Age,Week
0,ID00007637202177411956430,-0.533965,-0.810632,-0.843586,-0.43259,0.517799,1.662082,-0.030075
1,ID00009637202177434476278,1.150500,1.152282,0.709119,-0.43259,0.517799,0.249433,0.060150
2,ID00010637202177584971671,0.762048,0.342767,-0.288482,-0.43259,0.517799,-1.021951,0.000000


In [10]:
df.to_csv('data/train_base_3.csv', index=False)